In [1]:
#pip install ipywidgets matplotlib pandas pillow


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle
from PIL import Image
import json
import os
import ipywidgets as widgets
from IPython.display import display

# === Paths ===
CSV_PATH = "CDD-CESM/Radiology_hand_drawn_segmentations_v2.csv"
IMAGE_DIR = "CDD-CESM/PKG - CDD-CESM/CDD-CESM/Low energy images of CDD-CESM"

# === Load CSV ===
df = pd.read_csv(CSV_PATH)
df['#filename'] = df['#filename'].str.strip()  # Clean any stray spaces
images = df['#filename'].unique()

# === Group by image ===
grouped = df.groupby('#filename')

# === Dropdown Widget ===
dropdown = widgets.Dropdown(
    options=sorted(images),
    description='Image:',
    layout=widgets.Layout(width='400px')
)

def display_image_with_annotations(image_name):
    image_path = os.path.join(IMAGE_DIR, image_name)
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return

    # Load image
    img = Image.open(image_path)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img)
    ax.set_title(image_name)
    
    # Draw annotations
    try:
        rows = grouped.get_group(image_name)
        for _, row in rows.iterrows():
            shape = json.loads(row['region_shape_attributes'])
            if shape['name'] == 'ellipse':
                patch = Ellipse(
                    (shape['cx'], shape['cy']),
                    2 * shape['rx'],
                    2 * shape['ry'],
                    edgecolor='red',
                    facecolor='none',
                    linewidth=2
                )
                ax.add_patch(patch)
            elif shape['name'] == 'circle':
                patch = Circle(
                    (shape['cx'], shape['cy']),
                    shape['r'],
                    edgecolor='blue',
                    facecolor='none',
                    linewidth=2
                )
                ax.add_patch(patch)
    except Exception as e:
        print(f"Error parsing annotation: {e}")

    ax.axis("off")
    plt.show()

# === Callback ===
def on_image_change(change):
    display_image_with_annotations(change['new'])

# === Link the callback ===
dropdown.observe(on_image_change, names='value')

# === Display the GUI ===
display(dropdown)
# Initial display
display_image_with_annotations(dropdown.value)


Dropdown(description='Image:', layout=Layout(width='400px'), options=('P100_L_CM_CC.jpg', 'P100_L_CM_MLO.jpg',…

Image not found: CDD-CESM/PKG - CDD-CESM/CDD-CESM/Low energy images of CDD-CESM/P100_L_CM_CC.jpg
